In [ ]:
# Import app configuration
from typing import Dict
from config import APP_CONFIG

spark_config_dict: Dict[str, str] = APP_CONFIG.get('spark', dict())
input_config_dict: Dict[str, str] = APP_CONFIG.get('input', dict())
output_config_dict: Dict[str, str] = APP_CONFIG.get('output', dict())

SPARK_APP_NAME = spark_config_dict.get('name', 'spark-app')

In [ ]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.dataframe import DataFrame

os.environ['SPARK_MEM'] = spark_config_dict.get('memory', '24g')

spark_conf = SparkConf()
spark_conf.set('spark.driver.memory', spark_config_dict.get('driver.memory', '4g'))
spark_conf.set('spark.executor.memory', spark_config_dict.get('executor.memory', '5g'))
spark_conf.set('spark.executor.cores', spark_config_dict.get('executor.cores', '3'))
spark_conf.set('spark.executor.instances', spark_config_dict.get('executor.instances', '4'))
spark_conf.set('spark.dynamicAllocation.enabled', spark_config_dict.get('dynamicAllocation.enabled', 'false'))

# Configure and start new Spark Session
spark_session = (SparkSession.builder
                .appName(name=SPARK_APP_NAME)
                .master(master=spark_config_dict.get('master', 'local'))
                .config(conf=spark_conf)
                .getOrCreate())

spark_session.sparkContext.setLogLevel(spark_config_dict.get('logLevel', 'WARN'))

In [ ]:
requests_with_cap_df: DataFrame = spark_session.read.option('multiline', 'true').json(
    path='/Users/lap-mbp16-n01-0346/Documents/Sviluppo/Data Analysis/requests/request_with_cap2_PN-10033.json'
)

requests_with_pages_df: DataFrame = spark_session.read.option('multiline', 'true').json(
    path='/Users/lap-mbp16-n01-0346/Documents/Sviluppo/Data Analysis/requests/request_with_pages2_PN-10033.json'
)

matrice_costi_2023_pivot_df: DataFrame = spark_session.read.csv(
    path='resources/matrice_costi_2023_pivot.csv'
).withColumnsRenamed({
    '_c0': 'geokey', '_c1': 'product', '_c2': 'recapitista', '_c3': 'lotto', '_c4': 'costo_plico', '_c5': 'costo_foglio', '_c6': 'costo_demat', '_c7': 'min', '_c8': 'max', '_c9': 'costo', '_c10': 'costo_base_20gr'
})

In [ ]:
request_with_cap_and_pages_df: DataFrame = requests_with_pages_df \
    .join(requests_with_cap_df, on=['request']) \
    .select(requests_with_pages_df.iun, requests_with_pages_df.request, requests_with_pages_df.pages, requests_with_pages_df.product_type, requests_with_cap_df.cap)

request_with_cap_and_pages_df.show(n=100, truncate=False)

In [ ]:
weight_column: Column = 5 + (5 * request_with_cap_and_pages_df.pages)
min_weight: Column = when(weight_column <= 20, 1) \
    .when(weight_column <= 50, 21) \
    .when(weight_column <= 100, 51) \
    .when(weight_column <= 250, 101) \
    .when(weight_column <= 350, 251) \
    .when(weight_column <= 1000, 351) \
    .when(weight_column <= 2000, 1001)

request_with_cap_and_min_weight_df: DataFrame = request_with_cap_and_pages_df.select(
    request_with_cap_and_pages_df.iun,
    request_with_cap_and_pages_df.request,
    request_with_cap_and_pages_df.product_type,
    request_with_cap_and_pages_df.cap,
    request_with_cap_and_pages_df.pages,
    min_weight.alias('min_weight')
)

# request_with_cap_and_min_weight_df.show(n=100, truncate=False)

In [ ]:
requests_with_recapitista_df: DataFrame = request_with_cap_and_min_weight_df \
    .join(
        matrice_costi_2023_pivot_df, 
        on=(request_with_cap_and_min_weight_df.cap == matrice_costi_2023_pivot_df.geokey) & (request_with_cap_and_min_weight_df.product_type == matrice_costi_2023_pivot_df.product) & (request_with_cap_and_min_weight_df.min_weight == matrice_costi_2023_pivot_df.min),
        how='left'
).select(
    request_with_cap_and_pages_df.iun, 
    request_with_cap_and_pages_df.request, 
    request_with_cap_and_pages_df.cap,
    matrice_costi_2023_pivot_df.product, 
    matrice_costi_2023_pivot_df.lotto,
    matrice_costi_2023_pivot_df.recapitista,
    (request_with_cap_and_pages_df.pages - 1).alias('pages'),
    request_with_cap_and_min_weight_df.min_weight,
    matrice_costi_2023_pivot_df.costo_plico,
    matrice_costi_2023_pivot_df.costo_foglio,
    matrice_costi_2023_pivot_df.costo_demat,
    matrice_costi_2023_pivot_df.costo
)

requests_with_recapitista_df.show(n=100, truncate=False)

In [ ]:
# costo_plico + (analogico_totale_pagine-1)*costo_foglio
costo_consolidatore_column: Column = format_number(
    (requests_with_recapitista_df.costo_plico + (requests_with_recapitista_df.pages*requests_with_recapitista_df.costo_foglio)) / 100,
    2
).cast('float').alias('Costo_Consolidatore') 

# (costo + costo_demat)
costo_recapitista_column: Column = format_number(
    (requests_with_recapitista_df.costo + requests_with_recapitista_df.costo_demat) / 100,
    2
).cast('float').alias('Costo_Recapitista')

requests_with_recapitista_and_cost: DataFrame = requests_with_recapitista_df.select(
    requests_with_recapitista_df.iun.alias('IUN'), 
    requests_with_recapitista_df.request.alias('Richiesta'), 
    request_with_cap_and_pages_df.cap.alias('CAP'),
    lit('').alias('Stato_Estero'),
    matrice_costi_2023_pivot_df.product.alias('Prodotto_Postale'), 
    matrice_costi_2023_pivot_df.lotto.alias('Lotto'),
    matrice_costi_2023_pivot_df.recapitista.alias('Recapitista'),
    requests_with_recapitista_df.min_weight.alias('Scaglione_Minimo'),
    costo_consolidatore_column,
    costo_recapitista_column
)

requests_with_recapitista_and_cost.show(n=100, truncate=False)

In [ ]:
from utils.custom_data_frame_writer import CustomDataFrameWriter

# Write out dataframe
CustomDataFrameWriter.write(
    df=requests_with_recapitista_and_cost.repartition(1),
    output_name=SPARK_APP_NAME,
    output_folder=output_config_dict.get('path'),
    output_format=output_config_dict.get('format')
)

In [ ]:
"""WITH o_first10_iun_for_comune_details_final_costi AS (
    SELECT 
        iun,
        ente,
        id_ente,
        ordine_asseverazione,
        categoria_fatturazione,
        fatturazione_anno,
        fatturazione_mese,
        presaincarico_anno,
        presaincarico_mese,
        tipologia_notifica,
        prezzo_ente,
        analogico_prodotto,
        cast(analogico_totale_pagine as int) as analogico_totale_pagine,
        CASE
            WHEN analogico_totale_peso_gr BETWEEN 1 AND 20 THEN 1
            WHEN analogico_totale_peso_gr BETWEEN 21 AND 50 THEN 21
            WHEN analogico_totale_peso_gr BETWEEN 51 AND 100 THEN 51
            WHEN analogico_totale_peso_gr BETWEEN 101 AND 250 THEN 101
            WHEN analogico_totale_peso_gr BETWEEN 251 AND 350 THEN 251
            WHEN analogico_totale_peso_gr BETWEEN 351 AND 1000 THEN 351
            WHEN analogico_totale_peso_gr BETWEEN 1001 AND 2000 THEN 1001
        END as analogico_totale_peso_gr_min,
        CASE
            WHEN analogico_prodotto in ('RIS', 'RIR') THEN analogico_dest_stato
            ELSE analogico_dest_cap
        END as analogico_geokey
    FROM 
        o_first10_iun_for_comune_details_final
), o_first10_iun_for_comune_details_final_costi_details AS (
    SELECT 
        iun,
        ente,
        id_ente,
        ordine_asseverazione,
        categoria_fatturazione,
        fatturazione_anno,
        fatturazione_mese,
        presaincarico_anno,
        presaincarico_mese,
        tipologia_notifica,
        prezzo_ente,
        --analogico_prodotto
        lotto,
        recapitista,
        CASE tipologia_notifica
            WHEN 'ANALOGICO' THEN costo_plico + (analogico_totale_pagine-1)*costo_foglio
            ELSE 0
        END as costo_consolidatore,
        CASE tipologia_notifica
            WHEN 'ANALOGICO' THEN (costo + costo_demat)
            ELSE 0
        END as costo_recapitista,
        CASE tipologia_notifica
            WHEN 'ANALOGICO' THEN (analogico_totale_pagine-1)*costo_foglio
            ELSE 0
        END as delta_costo_consolidatore,
        CASE tipologia_notifica
            WHEN 'ANALOGICO' THEN (costo - costo_base_20gr)
            ELSE 0
        END as delta_costo_recapitista
    FROM
        o_first10_iun_for_comune_details_final_costi a
    LEFT JOIN matrice_costi_2023_pivot c
        ON a.analogico_prodotto=c.product AND a.analogico_geokey = c.geokey AND a.analogico_totale_peso_gr_min=c.min
)
SELECT
    ente,
    id_ente,
    ordine_asseverazione,
    iun,
    fatturazione_anno,
    fatturazione_mese,
    presaincarico_anno,
    presaincarico_mese,
    tipologia_notifica,
    prezzo_ente,
    lotto,
    recapitista,
    CAST(costo_consolidatore as INT),
    CAST(costo_recapitista as INT),
    CAST(delta_costo_consolidatore as INT),
    CAST(delta_costo_recapitista as INT)
FROM 
    o_first10_iun_for_comune_details_final_costi_details 
ORDER BY id_ente, ordine_asseverazione, 
    fatturazione_anno, 
    fatturazione_mese,
    presaincarico_anno,
    presaincarico_mese"""